In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os
os.chdir('..')

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个数学老师。"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
chain = prompt | model

In [3]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [4]:
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(return_messages=True, k=2)

In [5]:
from langchain_chinese import WithMemoryBinding

withMemoryChain = WithMemoryBinding(
  chain,
  get_session_history,
  memory,
  input_messages_key="input",
  history_messages_key="history",
)

In [6]:
withMemoryChain.invoke(
  {"input": "三角函数什么意思？?"},
  config={"configurable": {"session_id": "abc123"}},
)

AIMessage(content='三角函数是一类描述角度关系的数学函数。在三角函数中，最常见的包括正弦函数（sine）、余弦函数（cosine）、正切函数（tangent）、余切函数（cotangent）、正割函数（secant）和余割函数（cosecant）。这些函数可以用来描述直角三角形中角度和边长之间的关系，以及周期性现象中的波动特征。在数学和物理学中，三角函数被广泛应用于各种领域，如信号处理、振动学、天文学等。')

In [7]:
withMemoryChain.invoke(
  {"input": "正弦是什么?"},
  config={"configurable": {"session_id": "abc123"}},
)

AIMessage(content='正弦是三角函数中的一种，通常用符号sin表示。在直角三角形中，正弦函数代表着一个角的对边长度与斜边长度之比。具体来说，对于一个角为θ的直角三角形，正弦函数的定义为：\n\nsin(θ) = 对边 / 斜边\n\n其中，对边是与角θ相对的那条边的长度，斜边是直角三角形的斜边长度。正弦函数的取值范围在-1到1之间，当角度为0度时，正弦函数的值为0；当角度为90度时，正弦函数的值为1；当角度为180度时，正弦函数的值再次为0。正弦函数在周期性为360度的情况下，呈现出波动的特性，常被用于描述周期性现象中的变化规律。')

In [8]:
withMemoryChain.invoke(
  {"input": "小学会学到吗?"},
  config={"configurable": {"session_id": "abc123"}},
)

AIMessage(content='正弦函数通常不是小学阶段的数学课程内容。一般来说，正弦函数是在中学阶段的数学学习中出现的内容，特别是在学习三角函数时会详细讨论正弦函数的概念和性质。小学阶段的数学课程主要侧重于基本的数学概念、算术运算、几何形状等内容，一般不会涉及到三角函数这样的高阶数学概念。')

In [9]:
store['abc123'].messages

[HumanMessage(content='三角函数什么意思？?'),
 AIMessage(content='三角函数是一类描述角度关系的数学函数。在三角函数中，最常见的包括正弦函数（sine）、余弦函数（cosine）、正切函数（tangent）、余切函数（cotangent）、正割函数（secant）和余割函数（cosecant）。这些函数可以用来描述直角三角形中角度和边长之间的关系，以及周期性现象中的波动特征。在数学和物理学中，三角函数被广泛应用于各种领域，如信号处理、振动学、天文学等。'),
 HumanMessage(content='正弦是什么?'),
 AIMessage(content='正弦是三角函数中的一种，通常用符号sin表示。在直角三角形中，正弦函数代表着一个角的对边长度与斜边长度之比。具体来说，对于一个角为θ的直角三角形，正弦函数的定义为：\n\nsin(θ) = 对边 / 斜边\n\n其中，对边是与角θ相对的那条边的长度，斜边是直角三角形的斜边长度。正弦函数的取值范围在-1到1之间，当角度为0度时，正弦函数的值为0；当角度为90度时，正弦函数的值为1；当角度为180度时，正弦函数的值再次为0。正弦函数在周期性为360度的情况下，呈现出波动的特性，常被用于描述周期性现象中的变化规律。'),
 HumanMessage(content='小学会学到吗?'),
 AIMessage(content='正弦函数通常不是小学阶段的数学课程内容。一般来说，正弦函数是在中学阶段的数学学习中出现的内容，特别是在学习三角函数时会详细讨论正弦函数的概念和性质。小学阶段的数学课程主要侧重于基本的数学概念、算术运算、几何形状等内容，一般不会涉及到三角函数这样的高阶数学概念。')]

In [10]:
memory.buffer_as_messages

[HumanMessage(content='正弦是什么?'),
 AIMessage(content='正弦是三角函数中的一种，通常用符号sin表示。在直角三角形中，正弦函数代表着一个角的对边长度与斜边长度之比。具体来说，对于一个角为θ的直角三角形，正弦函数的定义为：\n\nsin(θ) = 对边 / 斜边\n\n其中，对边是与角θ相对的那条边的长度，斜边是直角三角形的斜边长度。正弦函数的取值范围在-1到1之间，当角度为0度时，正弦函数的值为0；当角度为90度时，正弦函数的值为1；当角度为180度时，正弦函数的值再次为0。正弦函数在周期性为360度的情况下，呈现出波动的特性，常被用于描述周期性现象中的变化规律。'),
 HumanMessage(content='小学会学到吗?'),
 AIMessage(content='正弦函数通常不是小学阶段的数学课程内容。一般来说，正弦函数是在中学阶段的数学学习中出现的内容，特别是在学习三角函数时会详细讨论正弦函数的概念和性质。小学阶段的数学课程主要侧重于基本的数学概念、算术运算、几何形状等内容，一般不会涉及到三角函数这样的高阶数学概念。')]